In [1]:
import src.delta_duck as dd, duckdb

In [2]:
### Delta
table = dd.getDeltaTable(table_path="data/data.delta")
print(table.version())

print(table.vacuum(1, dry_run=False, enforce_retention_duration=False))
print(table.history())

dd.postDeltaTable(table=table, path="data/data.delta", mode="overwrite", overwrite_schema=True)

table = dd.getDeltaTable(table_path="data/data.delta")
print(table.version())


0
[]
[{'timestamp': 1684182251898, 'operation': 'WRITE', 'operationParameters': {'mode': 'ErrorIfExists', 'partitionBy': '[]'}, 'isolationLevel': 'Serializable', 'isBlindAppend': True, 'engineInfo': 'Apache-Spark/3.2.2 Delta-Lake/2.0.1', 'operationMetrics': {'numFiles': '8', 'numOutputBytes': '868369134', 'numOutputRows': '128000000'}, 'txnId': '84a70ba7-c147-445d-9204-f630e7f82fb0'}]
1


In [3]:
### Polars
polar = dd.polarFromDelta(table)
print(polar.describe())

unpolar = dd.polarToDelta(polar, mode="overwrite", overwrite_schema=True)
print(unpolar.version())

shape: (9, 7)
┌────────────┬───────────────┬─────────────────┬───────────┬───────────┬───────────┬─────────────┐
│ describe   ┆ _c0           ┆ model_iteration ┆ model     ┆ id        ┆ state     ┆ converter   │
│ ---        ┆ ---           ┆ ---             ┆ ---       ┆ ---       ┆ ---       ┆ ---         │
│ str        ┆ f64           ┆ f64             ┆ str       ┆ str       ┆ f64       ┆ f64         │
╞════════════╪═══════════════╪═════════════════╪═══════════╪═══════════╪═══════════╪═════════════╡
│ count      ┆ 1.28e8        ┆ 1.28e8          ┆ 128000000 ┆ 128000000 ┆ 1.28e8    ┆ 1.28e8      │
│ null_count ┆ 0.0           ┆ 0.0             ┆ 0         ┆ 0         ┆ 0.0       ┆ 0.0         │
│ mean       ┆ 499999.5      ┆ 500.5           ┆ null      ┆ null      ┆ 44.128    ┆ 5024.0      │
│ std        ┆ 288675.135724 ┆ 288.674991      ┆ null      ┆ null      ┆ 32.188191 ┆ 3288.073005 │
│ min        ┆ 0.0           ┆ 1.0             ┆ default   ┆ a         ┆ 0.0       ┆ 1000.0    

In [4]:
### DuckDB
DB = duckdb.connect("data/database.duckdb")

duck = dd.duckFromDelta(table="tst", source=table, DB=DB)
print(DB.execute("DESCRIBE;"))

unduck = dd.duckToDelta(
    table="tst", 
    source=DB, 
    path="data/data.delta",
    mode="overwrite", 
    overwrite_schema=True)
print(unduck.version())

: 

: 